In [6]:
import pandas as pd
import time #to have today's date
timestr = time.strftime("%Y%m%d")

In [7]:
df_docs_newNum_topics_country_class_order = \
    pd.read_csv('.//results//master_dfs//docs_newNum_topics_country_class_keywords_text_q_'+timestr+'_V01.csv',
    sep=';', decimal=',', index_col=0)

## do dataframes with topics per country and actor group

In [8]:
#create list with all columns that should be added to the sum
list_all_columns = df_docs_newNum_topics_country_class_order.columns
list_to_sum = []
for x in list_all_columns:
    if 'perc_cont' in x:
        list_to_sum.append(x)

### per country

In [20]:
#show the topics per country
df_topics_country_gr = df_docs_newNum_topics_country_class_order.copy().\
    groupby(['Country'])[list_to_sum].sum().reset_index()

In [21]:
#order of countries: (based on country hierarchical clustering)
list_country_oder = ['DE', 'UK', 'CL', 'CO', 'US', 'KE', 'ZA', 'ID', 'IN', 'VN', 'PH', 'PK']
df_topics_country_gr = df_topics_country_gr.set_index('Country').reindex(list_country_oder).reset_index()

In [22]:
#create column with the sum of all percentages, e.g. the number of paragraphs (not 100%, because rounded)
df_topics_country_gr['perc_total'] = df_topics_country_gr[list_to_sum].sum(axis=1)

In [23]:
#make list with all the columns (to delete later on)
list_topics_columns_paras = list_to_sum.copy()
list_topics_columns_paras.append('perc_total')

In [24]:
#get for each country the percentage for each topic
#example: documents from country A consist to 10% of topic 1, to 60% topic 2, and 30% topic 3.
#now the dataframe has the percentage for each topic. Divide by the number of texts per country. 
df_country_topics_share = df_topics_country_gr.copy()
for x in df_country_topics_share.index:
    for y in list_to_sum:
        var_number = df_country_topics_share.loc[x,y]/df_country_topics_share.loc[x,'perc_total']*100
        df_country_topics_share.loc[x, y+'_total'] = round(var_number,1)
    
#drop and rename columns
n = 1
for x in list_topics_columns_paras:
    df_country_topics_share.drop(columns={x}, inplace=True)
    df_country_topics_share.rename(columns={x+'_total':n}, inplace=True)
    n+=1

In [26]:
df_country_topics_share.to_csv('.//results//master_dfs//country_topics_share_'+timestr+'_V01.csv',
    sep=';', decimal=',')

### per actor group

In [52]:
#show the topics per actors group
df_topics_class_gr = df_docs_newNum_topics_country_class_order.copy().\
    groupby(['class'])[list_to_sum].sum().reset_index()

In [53]:
#create column with the sum of all percentages, e.g. the number of paragraphs (not 100%, because rounded)
df_topics_class_gr['perc_total'] = df_topics_class_gr[list_to_sum].sum(axis=1)

In [54]:
#get for each actors group (Classification) the percentage for each topic
#example: documents from actor group A consist to 10% of topic 1, to 60% topic 2, and 30% topic 3.
#now the dataframe has the percentage for each topic. Divide by the number of texts per class. 
df_class_topics_share = df_topics_class_gr.copy()
for x in df_class_topics_share.index:
    for y in list_to_sum:
        var_number = df_class_topics_share.loc[x,y]/df_class_topics_share.loc[x,'perc_total']*100
        df_class_topics_share.loc[x, y+'_total'] = round(var_number,1)
    
#drop and rename columns
n = 1
for x in list_topics_columns_paras:
    df_class_topics_share.drop(columns={x}, inplace=True)
    df_class_topics_share.rename(columns={x+'_total':n}, inplace=True)
    n+=1

In [55]:
df_class_topics_share.to_csv('.//results//master_dfs//class_topics_share_'+timestr+'_V01.csv',
    sep=';', decimal=',')

### per country AND actor group

In [48]:
#calculate, for each actor group in each country, which topics appear in their documents?
#show the topics per country
df_topics_country_class_gr = df_docs_newNum_topics_country_class_order.copy().\
    groupby(['Country', 'class'])[list_to_sum].sum().reset_index()

In [49]:
#create column with the sum of all percentages, e.g. the number of paragraphs (not 100%, because rounded)
df_topics_country_class_gr['perc_total'] = df_topics_country_class_gr[list_to_sum].sum(axis=1)

In [50]:
#get for each actors group and country the percentage for each topic
#example: documents from actor group A in country i consist to 10% of topic 1, to 60% topic 2, and 30% topic 3.
#now the dataframe has the percentage for each topic. Divide by the number of texts per country+class. 
df_country_class_topics_share = df_topics_country_class_gr.copy()
for x in df_country_class_topics_share.index:
    for y in list_to_sum:
        var_number = df_country_class_topics_share.loc[x,y]/df_country_class_topics_share.loc[x,'perc_total']*100
        df_country_class_topics_share.loc[x, y+'_total'] = round(var_number,1)
    
#drop and rename columns
n = 1
for x in list_topics_columns_paras:
    df_country_class_topics_share.drop(columns={x}, inplace=True)
    df_country_class_topics_share.rename(columns={x+'_total':n}, inplace=True)
    n+=1

In [51]:
df_country_class_topics_share.to_csv('.//results//master_dfs//country_AND_class_topics_share_'+timestr+'_V01.csv',
    sep=';', decimal=',')

## calculate shares between countries for each topic, so each topic is 100%

### per country

In [37]:
#create column with the sum of all percentages, e.g. the number of paragraphs (not 100%, because rounded)
df_a = df_topics_country_gr.copy()
df_a = df_a.drop(columns=['perc_total'])
df_topics_country_gr_T = df_a.set_index('Country').transpose().reset_index().rename(columns={'index':'Topic'})

In [39]:
list_to_sum_T = list(df_topics_country_gr_T.columns)
list_to_sum_T.remove('Topic')

In [40]:
df_topics_country_gr_T['perc_total'] = df_topics_country_gr_T[list_to_sum_T].sum(axis=1)

In [41]:
#make list with all the columns (to delete later on)
list_countries_columns_paras_T = list_to_sum_T.append('perc_total')

In [42]:
df_country_topics_share_T = pd.DataFrame({'Topic':df_topics_country_gr_T['Topic']} )
var_num_count = len(list_to_sum_T)
for x in df_country_topics_share_T.index:
    var_share_100 = df_topics_country_gr_T.loc[x, 'perc_total']/100
    for y in list_to_sum_T:
         df_country_topics_share_T.loc[x, y] = round(df_topics_country_gr_T.loc[x,y]/var_share_100, 1)

In [43]:
#index not from 0-21, but 1-22
df_country_topics_share_T['idx'] = list(range(1,23))
df_country_topics_share_T.set_index('idx', inplace=True)

In [44]:
#remove the column with 'perc_total'
df_country_topics_share_T.drop(columns=['perc_total', 'Topic'], inplace=True)

In [45]:
df_country_topics_share_T.head(2)

,DE,UK,CL,CO,US,KE,ZA,ID,IN,VN,PH,PK
idx,,,,,,,,,,,,
1,9.7,6.2,9.6,6.3,4.9,9.8,13.5,7.7,14.8,6.3,9.1,2.1
2,10.6,2.3,9.9,19.7,3.2,5.1,5.4,6.2,19.5,4.1,10.0,3.9


In [46]:
#transpose again, so the dataframe has the same shape as the one without T (df_country_topics_share)
df_country_topics_share_T = df_country_topics_share_T.transpose().reset_index().rename(columns={'index':'Country'})

In [47]:
df_country_topics_share_T.to_csv('.//results//master_dfs//country_topics_share_T_'+timestr+'_V01.csv',
    sep=';', decimal=',')

### per actors group

In [56]:
#create column with the sum of all percentages, e.g. the number of paragraphs (not 100%, because rounded)
df_b = df_topics_class_gr.copy()
df_b = df_b.drop(columns=['perc_total'])
df_topics_class_gr_T = df_b.set_index('class').transpose().reset_index().rename(columns={'index':'Topic'})

list_to_sum_T = list(df_topics_class_gr_T.columns)
list_to_sum_T.remove('Topic')

df_topics_class_gr_T['perc_total'] = df_topics_class_gr_T[list_to_sum_T].sum(axis=1)

#make list with all the columns (to delete later on)
list_class_columns_paras_T = list_to_sum_T.append('perc_total')

In [57]:
df_class_topics_share_T = pd.DataFrame({'Topic':df_topics_class_gr_T['Topic']} )
var_num_count = len(list_to_sum_T)
for x in df_class_topics_share_T.index:
    var_share_100 = df_topics_class_gr_T.loc[x, 'perc_total']/100
    for y in list_to_sum_T:
         df_class_topics_share_T.loc[x, y] = round(df_topics_class_gr_T.loc[x,y]/var_share_100, 1)
            
#remove the column with 'perc_total'
df_class_topics_share_T.drop(columns=['perc_total', 'Topic'], inplace=True)


In [58]:
#index not from 0-21, but 1-22
df_class_topics_share_T['idx'] = list(range(1,23))
df_class_topics_share_T.set_index('idx', inplace=True)

In [59]:
#transpose again, so the dataframe has the same shape as the one without T (df_country_topics_share)
df_class_topics_share_T = df_class_topics_share_T.transpose().reset_index().rename(columns={'index':'class'})

In [60]:
df_class_topics_share_T.to_csv('.//results//master_dfs//class_topics_share_T_'+timestr+'_V01.csv',
    sep=';', decimal=',')

### per country AND actor group

In [61]:
#work on this one!!

In [62]:
#create column with the sum of all percentages, e.g. the number of paragraphs (not 100%, because rounded)
df_c= df_topics_country_class_gr.copy()
df_c= df_c.drop(columns=['perc_total'])
df_topics_country_class_gr_T = df_c.set_index(['Country', 'class']).transpose().reset_index().rename(columns={'index':'Topic'})

list_to_sum_T = list(df_topics_country_class_gr_T.columns)
list_to_sum_T.remove(('Topic', ''))

df_topics_country_class_gr_T['perc_total'] = df_topics_country_class_gr_T[list_to_sum_T].sum(axis=1)

#make list with all the columns (to delete later on)
list_country_class_columns_paras_T = list_to_sum_T.append('perc_total')

In [63]:
df_country_class_topics_share_T = pd.DataFrame({'Topic':df_topics_country_class_gr_T['Topic']} )
var_num_count = len(list_to_sum_T)
for x in df_country_class_topics_share_T.index:
    var_share_100 = df_topics_country_class_gr_T.loc[x, 'perc_total'][0]/100
    for y in list_to_sum_T:
        df_country_class_topics_share_T.loc[x, [y]] = round(df_topics_country_class_gr_T.loc[x,[y]]/var_share_100, 1)
        
#remove the column with 'perc_total'
df_country_class_topics_share_T.drop(columns=['perc_total', 'Topic'], inplace=True)

In [64]:
#index not from 0-21, but 1-22
df_country_class_topics_share_T['idx'] = list(range(1,23))
df_country_class_topics_share_T.set_index('idx', inplace=True)

In [65]:
#transpose
df_country_class_topics_share_T = df_country_class_topics_share_T.transpose().reset_index()

In [66]:
#now the 'index' column consists of tuples with the Country and the class. Split into columns
new_col_list = ['Country', 'class']
for n,col in enumerate(new_col_list):
    df_country_class_topics_share_T[col] = df_country_class_topics_share_T['index'].apply(lambda location: location[n])

df_country_class_topics_share_T = df_country_class_topics_share_T.drop('index',axis=1)

In [67]:
#change order of columns
list_a = list(df_country_class_topics_share_T.columns)
list_a.remove('Country')
list_a.remove('class')
list_b = ['Country', 'class'] + list_a
df_country_class_topics_share_T = df_country_class_topics_share_T[list_b]

In [68]:
df_country_class_topics_share_T.to_csv('.//results//master_dfs//country_AND_class_topics_share_T_'+timestr+'_V01.csv',
    sep=';', decimal=',')